In [ ]:
import tensorflow as tf
import numpy as np
import csv
import pandas as pd
import numpy as np
from scipy.ndimage.interpolation import shift
import matplotlib.pyplot as plt

In [ ]:
# Parameters
RANDOM_SEED = 42
tf.set_random_seed(RANDOM_SEED)

days_history = 4
features = 4
learningrate = 0.05
epochs = 100
percent_training = 0.8
mid_layer_size = 32

weight_dev = 1 # standard deviation of weights
#stock_type = "STL"
stock_type = "MHG"


In [ ]:
# Get dataset

def get_dataset(name):
    df = pd.read_csv(name, sep=';', index_col=0, encoding="Latin1")
    df = df[::-1]
    df.columns = ['Siste', 'Kjoper', 'Selger', 'Hoy', 'Lav', 'Totalt_omsatt_NOK', 'Totalt_antall_aksjer_omsatt', 'Antall_off_handler', 'Antall_handler_totalt', 'VWAP']
    return df


def get_currency_data(name):
    df = pd.read_csv(name, sep=';', index_col=0, encoding="Latin1")
    return df


def append_currency_to_stock_price_df(df_stock, df_currency):
    df_stock["Dollarkurs"] = df_currency["1 USD"]


stock = get_dataset("data\\" + stock_type + "_data.csv")
nok = get_currency_data("data\\Valutakurser.csv")

append_currency_to_stock_price_df(stock, nok)


In [ ]:
# Initialize labels

Up = np.roll(stock.Siste.diff(1),-1) > 0

n = len(Up)

train = np.zeros((n, 2))
train[:,0] = (Up == 0)
train[:,1] = (Up == 1)

cutoff = int(np.floor(n*percent_training))

Y_train = train[:cutoff,:]
Y_test = train[cutoff:,:]

Y_train = Y_train[days_history:]
print(Y_train.shape)
print(Y_test.shape)

In [ ]:
#print(stock.Siste)
print(stock.Siste[1])
print(stock.Siste[2])
print(np.roll(stock.Siste.diff(1),0)> 0)
print(np.roll(stock.Siste.diff(1),-1) > 0)


In [ ]:
# Initialize training data
input_dimension = features*days_history

Siste_all = np.zeros((n, days_history))
Siste = stock.Siste.values

Omsatte_all = np.zeros((n, days_history))
Omsatte = stock.Totalt_antall_aksjer_omsatt.values

Antall_handler_all = np.zeros((n, days_history))
Antall_handler = stock.Antall_handler_totalt.values

Valutakurs_all = np.zeros((n, days_history))
Valutakurs = stock.Dollarkurs.values


def normalize(vector, ref):
    ref_min = min(ref)
    ref_max = max(ref)
    vector -= ref_min
    return vector / (ref_max-ref_min)
    

for i in range(days_history):
    Siste_all[:,i] = np.roll(Siste, i)
    Omsatte_all[:,i] = np.roll(Omsatte, i)
    Antall_handler_all[:,i] = np.roll(Antall_handler, i)
    Valutakurs_all[:,i] = np.roll(Valutakurs, i)
   
Siste_all = normalize(Siste_all, Siste)    
Omsatte_all = normalize(Omsatte_all, Omsatte)
Antall_handler_all = normalize(Antall_handler_all, Antall_handler)
Valutakurs_all = normalize(Valutakurs_all, Valutakurs)

plt.plot(Siste_all[cutoff:])
plt.show()


X_data = Siste_all

X_data = np.append(X_data,Omsatte_all, 1)
X_data = np.append(X_data,Antall_handler_all, 1)
X_data= np.append(X_data,Valutakurs_all, 1)


plt.plot(X_data[days_history:])
plt.show()
print(X_data.shape)

X_train = X_data[:cutoff,:]
X_test = X_data[cutoff:,:]
X_train = X_train[days_history:]
print(X_train.shape)
print(X_test.shape)

In [ ]:
# Setup graph
def init_weights(shape):
    """ Weight initialization """
    weights = tf.random_normal(shape, stddev=weight_dev)
    return tf.Variable(weights)

# Layer's sizes
x_size = X_train.shape[1]  
h_size = mid_layer_size              
y_size = Y_train.shape[1] 

# Symbols
X = tf.placeholder("float", shape=[None, x_size])
y = tf.placeholder("float", shape=[None, y_size])

# Weight initializations
w_1 = init_weights((x_size, h_size))
w_2 = init_weights((h_size, y_size))

# Forward propagation
h    = tf.nn.tanh(tf.matmul(X, w_1)) 
yhat = tf.nn.softmax(tf.matmul(h, w_2))
predict = tf.argmax(yhat, axis=1)

# Backward propagation
cost    = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=yhat))
updates = tf.train.GradientDescentOptimizer(learningrate).minimize(cost)


In [ ]:
# Run graph
config = tf.ConfigProto(device_count = {'GPU': 0})
sess = tf.Session(config=config)
init = tf.global_variables_initializer()
sess.run(init)

for epoch in range(epochs):
    for i in range(len(X_train)):
        sess.run(updates, feed_dict={X: X_train[i: i + 1], y: Y_train[i: i + 1]})

    train_accuracy = np.mean(np.argmax(Y_train, axis=1) ==
                 sess.run(predict, feed_dict={X: X_train, y: Y_train}))
    test_accuracy  = np.mean(np.argmax(Y_test, axis=1) ==
                 sess.run(predict, feed_dict={X: X_test, y: Y_test}))
    train_negatives = np.average( sess.run(predict, feed_dict={X: X_train, y: Y_train}) == 0)
    train_positives = np.average( sess.run(predict, feed_dict={X: X_train, y: Y_train}) == 1)
    test_negatives = np.average( sess.run(predict, feed_dict={X: X_test, y: Y_test}) == 0)
    test_positives = np.average( sess.run(predict, feed_dict={X: X_test, y: Y_test}) == 1)
    print("Epoch = %d, tr_a = %.2f%%, te_a = %.2f%%. tr_pos: %.2f%%. tr_neg %.2f%%. te_pos: %.2f%%. te_neg %.2f%%."
          % (epoch + 1, 100. * train_accuracy, 100. * test_accuracy, 100. * train_positives, 100. * train_negatives, 100.* test_positives, 100.*test_negatives))


In [ ]:
test_results = (np.argmax(Y_test, axis=1) == sess.run(predict, feed_dict={X: X_test, y: Y_test}))
#print(sess.run(yhat, feed_dict={X: X_test, y: Y_test}))
predictions = sess.run(predict, feed_dict={X: X_test, y: Y_test})
labels = np.argmax(Y_test, axis=1)



#plt.subplot(2,1,1)
#plt.bar(ind,predictions[:50])
#plt.subplot(2,1,2)
#plt.bar(ind,labels[:50])
#plt.show()

start_index = 0
stop_index = 200
ind = range( stop_index - start_index)
print(predictions[start_index:stop_index])




test_data = Siste[cutoff + start_index:cutoff + stop_index]

val_max = max(test_data)
val_min = min(test_data)

plt.figure(figsize=(18,9))
plt.subplot(2,1,1)
plt.plot(test_data)
plt.bar(ind,labels[start_index:stop_index]*test_data)
plt.ylim([val_min*0.99,val_max*1.01])
plt.ylabel('Labels')
plt.subplot(2,1,2)
plt.plot(test_data)
plt.bar(ind,predictions[start_index:stop_index]*test_data)
plt.ylim([val_min*0.99,val_max*1.01])
plt.ylabel('Predictions')
plt.show()


test_data = Valutakurs[cutoff + start_index:cutoff + stop_index]

val_max = max(test_data)
val_min = min(test_data)

plt.figure(figsize=(18,9))
plt.subplot(2,1,1)
plt.plot(test_data)
plt.bar(ind,labels[start_index:stop_index]*test_data)
plt.ylim([val_min*0.99,val_max*1.01])
plt.ylabel('Labels')
plt.subplot(2,1,2)
plt.plot(test_data)
plt.bar(ind,predictions[start_index:stop_index]*test_data)
plt.ylim([val_min*0.99,val_max*1.01])
plt.ylabel('Predictions')
plt.show()




In [ ]:
sess.close()